In [ ]:
from sub_func import *


from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# 모델 및 토크나이저 불러오기
model_name = "snunlp/KR-FinBert-SC"  # Hugging Face에서 제공되는 KR-FinBert
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [31]:
pf_selection_report_format = """{
report: '(생성한 레포트 본문 전체)',
sector: '[레포트 내에 언급된 주요 섹터를 리스트 형식으로 반환]'
}
"""

pf_selection_system = f"""당신은 증권회사의 포트폴리오 전문가입니다.
주어진 다음 정보들은 이전 분기의 국제 뉴스, 거시경제 정보, 인덱스 지표들의 가격입니다.
이를 바탕으로 다음 분기에 주목해야 할 섹터를 정리해서 보고서 형식으로 출력해야 합니다.
보고서의 의견은 반드시 주어진 정보에 기반하여야 합니다.
보고서의 형식은 반드시 다음을 따라야 합니다. {pf_selection_report_format}

** Return only pure JSON format without any code block or delimiters. **
** Make sure that the response does not create JSON decode error. **
"""

pf_selection_prompt = f""""""

In [32]:
target_year = '2021'
target_quarter = 'Q1'

start_date = '20210101'
end_date = '20210331'

In [ ]:
# ==== 국제 뉴스 수집 ==== #
intl_news_title_list = list(intl_news_info(target_year, start_date, end_date)['news_title'])
pf_selection_prompt += f"""국제 뉴스 헤드라인: {intl_news_title_list}\n"""

# ==== 거시경제 정보 수집 ==== #
def create_macro_econ_dict(country, econ_item):
    result_dict = macro_econ_dict[country][econ_item].set_index('Date').to_dict('index')
    final_dict = {k: v[list(v.keys())[0]] for k, v in result_dict.items()}

    return final_dict

reports_dict = {}
macro_econ_dict = macro_econ_info(target_year, start_date, end_date)
country_list = list(macro_econ_dict.keys())

for country in country_list:
    target_country_dict = macro_econ_dict[country]
    econ_items = list(target_country_dict.keys())
    for econ_item in econ_items:
        final_dict = create_macro_econ_dict(country, econ_item)
        reports_dict[f"{country}_{econ_item}"] = final_dict
pf_selection_prompt += f"""국가별 거시경제 정보: {reports_dict}\n"""

# ==== 인덱스 가격 수집 ==== #
index_prices = {}
sector_list = [s for s in os.listdir('../store_data/raw/market_data/sector') if '코스피' not in s]

for sector in sector_list:
    # 토큰수 제한때문에 컬럼 선별해서 넣기...
    index_price = index_price_info(sector, start_date, end_date)[['Close', 'Transaction_Val','Market_Cap', 'RSI_14']]
    index_prices[sector] = index_price.T.to_dict()

pf_selection_prompt += f"""인덱스 가격 지표: {index_prices}\n"""

# 201375 -> 293650 -> 385768
stock_selection_response = to_GPT(pf_selection_system, pf_selection_prompt)

# 여기까지 해서, 투자할 섹터 결정
try:
    sample = eval(stock_selection_response['choices'][0]['message']['content'])
    pf_selection_report = sample['report']
    pf_selection_sector = sample['sector']
except Exception as e:
    print('-'*50)
    print(f"GPT 응답 구조가 올바르지 않음: {e}")

UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2021/2021_Inflation Rate.csv'


### 투자 섹터와 연관있는 종목을 찾기 위해 재무 보고서와 임베딩 비교

In [57]:
target_df = reports_info('005930', target_year, target_quarter)
target_report = target_df['1. 회사의 개요.csv'][0]

In [64]:
# 문장을 벡터로 변환하는 함수 정의
def get_sentence_embedding(sentence):
    # 문장 토큰화
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # 모델에서 출력 얻기
    with torch.no_grad():
        outputs = model(**inputs)
    # [CLS] 토큰의 벡터를 사용
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Batch, CLS token, Hidden size
    return cls_embedding

# 코사인 유사도 계산 함수
def calculate_cosine_similarity(embedding1, embedding2):
    # 텐서를 NumPy 배열로 변환 후 코사인 유사도 계산
    embedding1 = embedding1.cpu().numpy()
    embedding2 = embedding2.cpu().numpy()
    return cosine_similarity(embedding1, embedding2)[0][0]

def finBERT_cosine_similarity(sentence, keyword):
    """
    KR-FinBERT으로 임베딩해서 재무보고서와 섹터간의 유사도 분석
    """
    # 문장을 임베딩으로 변환
    embedding = get_sentence_embedding(sentence)
    keyword_embedding = get_sentence_embedding(keyword)

    # 코사인 유사도 계산
    similarity = calculate_cosine_similarity(embedding, keyword_embedding)

    return similarity

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [65]:
similarity_dict = {}
for sector in pf_selection_sector:
    similarity_dict[sector] = finBERT_cosine_similarity(target_report, sector)

In [66]:
similarity_dict

{'반도체': 0.30658692,
 '전기차 및 배터리': 0.26650867,
 '친환경 및 ESG': 0.80780256,
 '유통업': 0.55815053,
 '금융업': 0.7398901}